### Import dependencies

In [83]:
from subprocess import run, PIPE
import os
# from flask import logging, Flask, render_template, request
import logging
import random
import librosa
import itertools
import tensorflow as tf
from keras.models import model_from_json
from keras import backend as K
from keras.activations import relu
# import soundfile
import soundfile as sf
from keras.preprocessing.sequence import pad_sequences
from spell_correction import correction

from numpy.lib.stride_tricks import as_strided
import math
# import kenlm
import numpy as np

### Declearation of our Urdu Vocablary

In [84]:

char_map_str = """
<SPACE> 0
ا 1
آ 2
ب 3
پ 4
ت 5
ٹ 6
ث 7
ج 8
چ 9
ح 10
خ 11
د 12
ڈ 13
ذ 14
ر 15
ڑ 16
ز 17
ژ 18
س 19
ش 20
ص 21
ض 22
ط 23
ظ 24
ع 25
غ 26
ف 27
ق 28
ک 29
گ 30
ل 31
م 32
ن 33
ں 34
و 35
ہ 36
ھ 37
38 ء
ئ 39
ی 40
ے  41
'

"""

char_map = {}
index_map = {}

for line in char_map_str.strip().split('\n')[0:-1]:
    try:
        ch, index = line.split()
    #     print('ch ',ch,' index ',index)
        char_map[ch.strip()] = int(index)
        index_map[int(index)] = ch.strip()
    except:
        # print('ch ',ch,' index ',index)
        char_map[index.strip()] = int(ch) 
        index_map[int(ch)] = index.strip()
       
#         break

index_map[0] = ' '

def int_to_text_sequence(seq):
   
    text_sequence = []
    for c in seq:
        if c == 42: #ctc/pad char
            ch = ''
        else:
            ch = index_map[c]
        text_sequence.append(ch)
    return text_sequence

#### functions to extract audio features

In [85]:

def clipped_relu(x):
    return relu(x, max_value=20)
def make_specto_shape(filename, padlen=778):
    r = spectrogram_from_file(filename)
    print("shape of specto ",r.shape)
    t = np.transpose(r)  # 2D array ->  spec x timesamples
    X = pad_sequences(t, maxlen=padlen, dtype='float', padding='post', truncating='post').T

    return X  # MAXtimesamples x specto {max x 161}
def get_max_specto_time(filename):
    r = spectrogram_from_file(filename)
    # print(r.shape)
    return r.shape[0]  #
def spectrogram(samples, fft_length=256, sample_rate=2, hop_length=128):
    """
    Compute the spectrogram for a real signal.
    The parameters follow the naming convention of
    matplotlib.mlab.specgram

    Args:
        samples (1D array): input audio signal
        fft_length (int): number of elements in fft window
        sample_rate (scalar): sample rate
        hop_length (int): hop length (relative offset between neighboring
            fft windows).

    Returns:
        x (2D array): spectrogram [frequency x time]
        freq (1D array): frequency of each row in x

    Note:
        This is a truncating computation e.g. if fft_length=10,
        hop_length=5 and the signal has 23 elements, then the
        last 3 elements will be truncated.
    """
    assert not np.iscomplexobj(samples), "Must not pass in complex numbers"

    window = np.hanning(fft_length)[:, None]
    window_norm = np.sum(window**2)

    # The scaling below follows the convention of
    # matplotlib.mlab.specgram which is the same as
    # matlabs specgram.
    scale = window_norm * sample_rate

    trunc = (len(samples) - fft_length) % hop_length
    x = samples[:len(samples) - trunc]

    # "stride trick" reshape to include overlap
    nshape = (fft_length, (len(x) - fft_length) // hop_length + 1)
    nstrides = (x.strides[0], x.strides[0] * hop_length)
    x = as_strided(x, shape=nshape, strides=nstrides)

    # window stride sanity check
    assert np.all(x[:, 1] == samples[hop_length:(hop_length + fft_length)])

    # broadcast window, compute fft over columns and square mod
    x = np.fft.rfft(x * window, axis=0)
    x = np.absolute(x)**2

    # scale, 2.0 for everything except dc and fft_length/2
    x[1:-1, :] *= (2.0 / scale)
    x[(0, -1), :] /= scale

    freqs = float(sample_rate) / fft_length * np.arange(x.shape[0])

    return x, freqs

def spectrogram_from_array(audio,sample_rate, step=10, window=20, max_freq=None,
                          eps=1e-14):
    """ Calculate the log of linear spectrogram from FFT energy
    Params:
        filename (str): Path to the audio file
        step (int): Step size in milliseconds between windows
        window (int): FFT window size in milliseconds
        max_freq (int): Only FFT bins corresponding to frequencies between
            [0, max_freq] are returned
        eps (float): Small value to ensure numerical stability (for ln(x))
    """
#     with soundfile.SoundFile(filename) as sound_file:
#     audio = sound_file.read(dtype='float32')
#     audio,sample_rate = librosa.load(filename,sr=16000)
#     sample_rate = sound_file.samplerate
    if audio.ndim >= 2:
        audio = np.mean(audio, 1)
    if max_freq is None:
        max_freq = sample_rate / 2
    if max_freq > sample_rate / 2:
        raise ValueError("max_freq must not be greater than half of "
                         " sample rate")
    if step > window:
        raise ValueError("step size must not be greater than window size")
    hop_length = int(0.001 * step * sample_rate)
    fft_length = int(0.001 * window * sample_rate)
    pxx, freqs = spectrogram(
        audio, fft_length=fft_length, sample_rate=sample_rate,
        hop_length=hop_length)
    ind = np.where(freqs <= max_freq)[0][-1] + 1
    return np.transpose(np.log(pxx[:ind, :] + eps))

def spectrogram_from_file(filename, step=10, window=20, max_freq=None,
                          eps=1e-14):
    """ Calculate the log of linear spectrogram from FFT energy
    Params:
        filename (str): Path to the audio file
        step (int): Step size in milliseconds between windows
        window (int): FFT window size in milliseconds
        max_freq (int): Only FFT bins corresponding to frequencies between
            [0, max_freq] are returned
        eps (float): Small value to ensure numerical stability (for ln(x))
    """
#     with soundfile.SoundFile(filename) as sound_file:
#     audio = sound_file.read(dtype='float32')
    audio,sample_rate = librosa.load(filename,sr=None)
    audio = librosa.resample(audio,sample_rate,16000)
    sample_rate=16000
#     sample_rate = sound_file.samplerate
    if audio.ndim >= 2:
        audio = np.mean(audio, 1)
    if max_freq is None:
        max_freq = sample_rate / 2
    if max_freq > sample_rate / 2:
        raise ValueError("max_freq must not be greater than half of "
                         " sample rate")
    if step > window:
        raise ValueError("step size must not be greater than window size")
    hop_length = int(0.001 * step * sample_rate)
    fft_length = int(0.001 * window * sample_rate)
    pxx, freqs = spectrogram(
        audio, fft_length=fft_length, sample_rate=sample_rate,
        hop_length=hop_length)
    ind = np.where(freqs <= max_freq)[0][-1] + 1
    return np.transpose(np.log(pxx[:ind, :] + eps))

def model_input(filename):
    x_val = [get_max_specto_time(filename)]
    max_val = max(x_val)
    # print("Max batch time value is:", max_val)
    X_data = np.array([make_specto_shape(filename, padlen=max_val)])
    with session.as_default():
        with session.graph.as_default():
            output = report([X_data])[0]
    #for i in output:
#     out = output[i]
    best = list(np.argmax(output[0], axis=1))

    #     if merge_chars:
    merge = [k for k,g in itertools.groupby(best)]
    
    #print#()
    return "".join(int_to_text_sequence(merge))
config = tf.compat.v1.ConfigProto(
    device_count={'GPU': 1},
    intra_op_parallelism_threads=1,
    allow_soft_placement=True
)

config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.6

session = tf.compat.v1.Session(config=config)

K.set_session(session)
                        
path='/home/baeg/Desktop/SpeakCv/Latest_Model/model'
jsonfilename = path+".json"
weightsfilename = path+".h5"

json_file = open(jsonfilename, 'r')
loaded_model_json = json_file.read()
json_file.close()

# K.set_learning_phase(1)

loaded_model = model_from_json(loaded_model_json,custom_objects={"clipped_relu": clipped_relu})

# load weights into loaded model
loaded_model.load_weights(weightsfilename)
loaded_model._make_predict_function()
y_pred = loaded_model.get_layer('ctc').input[0]
# print('done1')
input_data = loaded_model.get_layer('the_input').input
# print('done2')
K.set_learning_phase(0)
# print('done3')
report = K.function([input_data, K.learning_phase()], [y_pred])
# print('done4')

# def audio():
#     n = random.randint(0,9)
#     with open('tmp/audio'+str(n)+'.wav', 'wb') as f:
#         f.write(request.data)
    
#     #audio,sample_rate=librosa.load('tmp/audio'+str(n)'.wav',sr=16000)
#     print('saved as tmp/audio'+str(n)+'.wav')
#     #proc = run(['ffprobe', '-of', 'default=noprint_wrappers=1', 'tmp/audio'+str(n)+'.wav'],stderr=PIPE)
#     transcript=model_input('tmp/audio'+str(n)+'.wav')
#     #clean = run(['ffmpeg -i tmp/audio.wav -af "highpass=f=200, lowpass=f=3000" tmp/audio-clean.wav'],stderr=PIPE)

#     return transcript

def wavCorrector():
    fileNames=['name','age',"address","phone","education","year","profession","experience"]
    fileCount= len(fileNames)
    i=0
    for f in fileNames:
        i+=1
        loc="/home/baeg/Desktop/SpeakCv/Audios/recordedAudio ("+str(i)+").wav"
        if (os.path.exists(loc)==False):
            return False
        path=os.path.join(loc)
        x,_ = librosa.load(path, sr=16000)
        path=os.path.join('/home/baeg/Desktop/SpeakCv/Audios/', f+'.wav')
        os.remove(loc)
        sf.write(path, x, 16000) 
        print(i)
        
    return True
'''
if audio exists, correctly named copy, delete previous
else break
'''
wavCorrector()

def asr(location):
    # ***** Correcting sample rate of .wav files before sending to asr  ***** 
#     x,_ = librosa.load(location, sr=16000)
#     sf.write(location, x, 16000)
    
    # ***** CALLING ASR TO GET GENERATE TRANSCRIOTS  ***** 
    transcript = model_input(location)
    
    print("Original:", transcript)
    transcript = correction(transcript)
    print("LM corrected:", transcript)
    return transcript


### Defining functions for keyword correction

In [86]:

################################## Keyword functions   
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

def replaceUnicode(words):
    for i in range(len(words)):
           x=words[i].replace('\u202c', '')
           words[i] = x.replace('\u202b', '')
    return words
def city_Dist(sentence):
    words =["کراچی", "لاہور", "فیصل", "آباد", "راول", "پنڈی","راولپنڈی", "گوجرانوالہ" ,"پشاور", "ملتان", "حیدرآباد" ,"اسلام آباد","اسلام", "آباد", "کوئٹہ" ,"بہاولپور" ,"بہاولپور", "سرگودھا", "سیالکوٹ"]
    words=replaceUnicode(words)
    sentence = nltk.tokenize.word_tokenize(sentence)
    sentence
    listed = list()
    for word in words:
        for i in range(len(sentence)):
          ed = nltk.edit_distance(word,sentence[i])
          listed.append(tuple([word,i, ed]))
          listed.sort(key=lambda x:x[2])
    nearest=listed[0][0]
    index=listed[0][1]
    if listed[1][1]-listed[0][1]==1: #abs(
            nearest=listed[0][0]+listed[1][0]
    return nearest,index 

def name_Dist(sentence):
    words =["جواد", "حسن", "شفق" ,"ریاض" ,"بھٹی", "اعجاز" ,"احمد" ,"نورین" ,"جمیل" ,"زینب","عابد", "عمیر" ,"ارشد" ,"مدیحہ" ,"عمر", "محمد", "توقیر", "طلحہ", "مجاہد" ,"حمزہ" ,"علی" ,"علینہ", "بیگ" ,"جواد", "حسن", "شفق" ,"ریاض" ,"بھٹی" ,"اعجاز" ,"احمد", "نورین" ,"جمیل", "زینب" ,"عابد", "عمیر", "ارشد", "مدیحہ" ,"عمر", "محمد", "توقیر" ,"طلحہ", "مجاہد","حمزہ" ,"علی" ,"علینہ" ,"بیگ" ,"رباب", "عباس" , "ادریس" ,"خان" ,"سونیا", "عدنان", "کاشف", "غلام", "قادر", "دعا" ,"ساجدہ", "عزیر" ,"ہما", "شریف", "شوکت", "ماہین", "الف", "خان" ,"عبد" , "عتیق", "الرحمن" ,"شان", "لائبہ", "رضوان", "مرتضیٰ", "عدینہ","حنا", "زین", "مناہیل", "نور", "زرک" ,"آمنہ" ,"سدرہ", "ہنزلہ" ,"الیشبہ" ,"روہیل" ,"شیزہ", "فرخ", "صائمہ" ,"آیان" ,"سحر", "شازیب", "رؤف", "شاہ", "مدیحہ", "جمشید" ,"عریج",
    "علی" ,"حسنین", "نعمان", "جاوید" ,"منال" ,"اروبا", "لقمان" ,"ردا", "عاصم" ,"خدیجہ" ,"مدثر", "مریم" ,"رحیم","سرمد" ,"منیب" ,"نائلہ" ,"زوہیب","سارہ" ,"شہر", "یار" ,"اینی", "معاذ", "کشاف" ,"گل", "شیر" ,"ہمنا", "عاقب", "امان" ,"موحد" ,"فضل", "ایسان" ,"طارق" ,"فہد","توقیر", "ضیا", "مصطفیٰ" ,"خواجہ", "شارق", "محمد", "خان", "شہزاد", "حسام", "ولید", "عقبال", "ابرار", "حسنین", "وقار", "فیصل" ,"آصف", "فاطمہ",
    "زینب" "احمد", "عثمان", "انم" , "شیخ" ,"ماہرخ" ,"بلال" ,"رمشا" ,"زرش", "وقاص" ,"صدف" ,"راشد", "علیا" ,"سلمان", "آمنہ", "ذیشان" ,"تابندہ" ,"سعد", "ابراہیم" ,"زاہد" ,"راجہ", "آمنہ" ,"عائشہ", "اسماعیل" ,"حماد" ,"سیف" ,"احمد" ,
    "یاسر" ,"مزمل" ,"ملک" ,"ایشال"  ,"وسیم" ,"عامر" ,"رابعہ" ,"حسین" ,"سہیل" ,"شیراز" ,"عائزہ" ,"عامر", "عنایہ" ,"کامران", "فزا"]

    words=replaceUnicode(words)
    listed = list()
    for word in words:
#         for i in range(len(sentence)):
          ed = similar(word,sentence)
          listed.append(tuple([word,sentence, ed]))
          listed.sort(key=lambda x:x[2] , reverse=True)
    nearest=listed[0][0]
    return nearest
def set_name(sentence):
  tokens = nltk.tokenize.word_tokenize(sentence)
  sentence=tokens
  name=[]
  for i in range(len(tokens)):
    if (tokens[i]=='میرا'):
      i+=1
      if (tokens[i]=='نام'):
        i+=1
        name=tokens[i:]
        for n in range(i,len(tokens)):
            if (sentence[i]!="ہے"):
                x=name_Dist(sentence[i])
                sentence[i]=x
            i+=1
    elif (tokens[i]=='میں'):
      i+=1
      name=tokens[i:]
      for n in range(i,len(tokens)):
            if (sentence[i]!="ہو" and sentence[i]!="ہوں" ):
                x=name_Dist(sentence[i])
                sentence[i]=x 
  str1=" "
  if len(name)==0:
    str1='None'
  else:
    str1=str1.join(sentence)
    
  return str1

def set_Address(s):
    tokens = nltk.tokenize.word_tokenize(s)
    sentence=tokens
    words =["میرا‬","تعلق‬","سے‬","ہے‬","رہتا‬","ہوں","ہو","میں","کراچی", "لاہور", "فیصل", "آباد", "راول", "پنڈی","راولپنڈی", "گوجرانوالہ" ,"پشاور", "ملتان", "حیدرآباد" ,"اسلام آباد","اسلام", "آباد", "کوئٹہ" ,"بہاولپور" ,"بہاولپور", "سرگودھا", "سیالکوٹ"]
    words=replaceUnicode(words)
    for i  in range(len(tokens)):
        listed = list()
        for word in words:
    #         for i in range(len(sentence)):
              ed = similar(word,sentence[i])
              listed.append(tuple([word,sentence[i], ed]))
              listed.sort(key=lambda x:x[2] , reverse=True)
        nearest=listed[0][0]
        sentence[i]=nearest
    str1=" "
    if len(sentence)==0:
        str1='None'
    else:
        str1=str1.join(sentence)
    return str1
def set_Phone(s):
    tokens = nltk.tokenize.word_tokenize(s)
    sentence=tokens
    words = ["صفر","ایک","دو","تین","چار","پانچ","چھ","سات","آٹھ","نو","میرا","فون" ,"نمبر","زیرو","ون", "ٹو" ,"تھری" ,"فور", "فائیو" ,"سکس" ,"سیون", "آٹھ", "نائن","ہے", "یہ"]

    words=replaceUnicode(words)
    for i  in range(len(tokens)):
        listed = list()
        for word in words:
    #         for i in range(len(sentence)):
              ed = similar(word,sentence[i])
              listed.append(tuple([word,sentence[i], ed]))
              listed.sort(key=lambda x:x[2] , reverse=True)
        nearest=listed[0][0]
        sentence[i]=nearest
    str1=" "
    if len(sentence)==0:
        str1='None'
    else:
        str1=str1.join(sentence)
    return str1

##############################################################

### Defining functions for keyword extraction and CV generation

In [87]:
import os
import time
########
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 
##########
import re
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

from google_trans_new import google_translator 

[nltk_data] Downloading package punkt to /home/baeg/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/baeg/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [88]:
'''
    is_number , text2int functions are used to convert number words to numbers
    taken from https://stackoverflow.com/questions/493174/is-there-a-way-to-convert-number-words-to-integers
'''
def is_number(x):
    if type(x) == str:
        x = x.replace(',', '')
    try:
        float(x)
    except:
        return False
    return True

def text2int (textnum, numwords={}):
    units = [
        'zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight',
        'nine', 'ten', 'eleven', 'twelve', 'thirteen', 'fourteen', 'fifteen',
        'sixteen', 'seventeen', 'eighteen', 'nineteen',
    ]
    tens = ['', '', 'twenty', 'thirty', 'forty', 'fifty', 'sixty', 'seventy', 'eighty', 'ninety']
    scales = ['hundred', 'thousand', 'million', 'billion', 'trillion']
    ordinal_words = {'first':1, 'second':2, 'third':3, 'fifth':5, 'eighth':8, 'ninth':9, 'twelfth':12}
    ordinal_endings = [('ieth', 'y'), ('th', '')]

    if not numwords:
        numwords['and'] = (1, 0)
        for idx, word in enumerate(units): numwords[word] = (1, idx)
        for idx, word in enumerate(tens): numwords[word] = (1, idx * 10)
        for idx, word in enumerate(scales): numwords[word] = (10 ** (idx * 3 or 2), 0)

    textnum = textnum.replace('-', ' ')

    current = result = 0
    curstring = ''
    onnumber = False
    lastunit = False
    lastscale = False

    def is_numword(x):
        if is_number(x):
            return True
        if word in numwords:
            return True
        return False

    def from_numword(x):
        if is_number(x):
            scale = 0
            increment = int(x.replace(',', ''))
            return scale, increment
        return numwords[x]

    for word in textnum.split():
        if word in ordinal_words:
            scale, increment = (1, ordinal_words[word])
            current = current * scale + increment
            if scale > 100:
                result += current
                current = 0
            onnumber = True
            lastunit = False
            lastscale = False
        else:
            for ending, replacement in ordinal_endings:
                if word.endswith(ending):
                    word = "%s%s" % (word[:-len(ending)], replacement)

            if (not is_numword(word)) or (word == 'and' and not lastscale):
                if onnumber:
                    # Flush the current number we are building
                    curstring += repr(result + current) + " "
                curstring += word + " "
                result = current = 0
                onnumber = False
                lastunit = False
                lastscale = False
            else:
                scale, increment = from_numword(word)
                onnumber = True

                if lastunit and (word not in scales):                                                                                                                                                                                                                                         
                    # Assume this is part of a string of individual numbers to                                                                                                                                                                                                                
                    # be flushed, such as a zipcode "one two three four five"                                                                                                                                                                                                                 
                    curstring += repr(result + current)                                                                                                                                                                                                                                       
                    result = current = 0                                                                                                                                                                                                                                                      

                if scale > 1:                                                                                                                                                                                                                                                                 
                    current = max(1, current)                                                                                                                                                                                                                                                 

                current = current * scale + increment                                                                                                                                                                                                                                         
                if scale > 100:                                                                                                                                                                                                                                                               
                    result += current                                                                                                                                                                                                                                                         
                    current = 0                                                                                                                                                                                                                                                               

                lastscale = False                                                                                                                                                                                                              
                lastunit = False                                                                                                                                                
                if word in scales:                                                                                                                                                                                                             
                    lastscale = True                                                                                                                                                                                                         
                elif word in units:                                                                                                                                                                                                             
                    lastunit = True

    if onnumber:
        curstring += repr(result + current)

    return curstring
#################################### Keyword extraction functions ############################################
def get_name(sentence):
  tokens = nltk.tokenize.word_tokenize(sentence)
  name=[]
  for i in range(len(tokens)):
    #my name is
    if (tokens[i].lower()=='name'):
      i+=1
      if (tokens[i].lower()=='is'):
        i+=1
      name=tokens[i:]
    #I am hanan
    elif (tokens[i].lower()=='i'):
      i+=1
      if (tokens[i].lower()=='am'):
        i+=1
      name=tokens[i:]
  name[:2]
  str1=" "
  if len(name)==0:
    str1='None'
  else:
    str1=str1.join(name)
  return str1


def get_age(sentence):
  tokens = nltk.tokenize.word_tokenize(sentence)
  age=[]

  doc=nlp(sentence)
  for X in doc.ents:
    if(X.label_=='DATE'):
      age.append(X.text)

  if(len(age)==0):
    age=['None']
    
  str1=" "
  if len(age)==0:
    str1='None'
  else:
    str1=str1.join(age)
  return str1

def get_phone(sentence):
  num = ""
  for c in sentence:
      if c.isdigit():
          num = num + c
  if num=="":
    num='03311234567'
  return num


def get_address(sentence):
  tokens = nltk.tokenize.word_tokenize(sentence)
    #what if users says something diff like mera ghar karachi main hain lekin islamabad main kaam kr raha hun  ("I live in Rawalpindi")
    #POS tagger , NER
  addr=[]
  for i in range(len(tokens)):

    #I live in Faisalabad
    if (tokens[i].lower()=='live'):
      i+=1
      if (tokens[i].lower()=='in'):
        i+=1
      addr=tokens[i:]

    #I belong to 
    elif (tokens[i].lower()=='belong'):
      i+=1
      if (tokens[i].lower()=='to'):
        i+=1
      addr=tokens[i:]

    #I am from 
    elif (tokens[i].lower()=='from'):
      i+=1
      addr=tokens[i:]
    else:
      addr=tokens[i:]

  addr[:2]
  str1=" "
  if len(addr)==0:
    str1='None'
  else:
    str1=str1.join(addr)
  return str1


def get_education(sentence):
  year=''
  s=text2int(sentence) +' '     #convert year in words to number                                  
  yr=re.findall("(\d{4})\D",s)
  if (len(yr)!=0):      # IF THERE is any year mentioned, get it
    year=yr[0]

  tokens = nltk.tokenize.word_tokenize(sentence)
  EDUCATION = ['1st', '2nd', '3rd', '4th', '5th', '6th', '7th', '8th', '9th', '10th','11th','12th',
              'first','second','third','fourth','fifth','sixth','seventh','eight','ninth','tenth','elevnth','twelfth',
              'inter','middle','intermediate','matriculation','matric','ICS','Diploma','FSC','FA',
              'BA','BCom','B.Com',  'BTECH', 'B.TECH' , 'graduate', 'SSC', 'HSC' ,'illiterate' ]
  edu=[]
  for i in range(len(tokens)):
    x=tokens[i].lower()
    for i in range(len(EDUCATION)):
      c=EDUCATION[i].lower()
      if (nltk.edit_distance(c , x )<=1):
          if (i<24):
            c+=" grade"
          edu.append(c)
          break
  if len(edu)==0:
    str1='None'
  else:
    str1=str(edu[0])
  return str1

def get_year(sentence):
    year=''
    s=text2int(sentence) +' '     #convert year in words to number                                  
    yr=re.findall("(\d{4})\D",s)
    if (len(yr)!=0):      # IF THERE is any year mentioned, get it
         year=str(max(yr))

    return year


def get_profession(sentence):
  tokens = nltk.tokenize.word_tokenize(sentence)
  proffession=["delivery","carpenter", "driver","electrician","plumber"]
  #"I know plumber work"
  #"I am a carpenter"
  prof=[]
  for i in range(len(tokens)):
    #I live in Faisalabad
    x=tokens[i].lower()
    for c in proffession:
      if (nltk.edit_distance(c , x )<3):
          if (c=="delivery"):
            c="delivery boy"
          prof.append(c)
  str1=" "
  if len(prof)==0:
    str1='None'
  else:
    str1=str1.join(prof) 
  return str1


def get_experience(line):
  tokens = nltk.tokenize.word_tokenize(line)
  exp=[]
  doc=nlp(line)
  for X in doc.ents:
    if(X.label_=='DATE'):
      exp.append(X.text)

  if(len(exp)==0):
    exp=['None']
    
  str1=" "
  if len(exp)==0:
    str1='None'
  else:
    str1=str1.join(exp) 
  
  return str1


#################################### Keyword placement function ############################################
from docx import Document
def Place_Keywords(arr,path):
  document = Document(path)
  for paragraph in document.paragraphs:
      if 'nm_x' in paragraph.text:
          #name,age,address,phone,education, year,profession,experience
          #0    1    2        3      4        5     6         7
          paragraph.text = arr[0]
      elif 'ad_x' in paragraph.text:
          paragraph.text = arr[2]
      elif 'ph_x' in paragraph.text:
          paragraph.text = arr[3]
      elif 'ag_x' in paragraph.text:
          paragraph.text = arr[1]
      elif 'Prof_x' in paragraph.text:
          paragraph.text = arr[6]
      elif 'exp_x' in paragraph.text:
          paragraph.text = arr[7]
      elif 'ed_x' in paragraph.text:
          # print(paragraph.text)
          paragraph.text = arr[4]
      elif 'Yr_x' in paragraph.text:
          paragraph.text = arr[5]
      document.save('/home/baeg/Desktop/SpeakCv/CV/MY_CV.docx')
  return

def ifNone(str):
    if len(str)==0:
        return 'None'
    else:
        return str


# GenerateCV function does the following tasks

*   generates text
*   Translates it
*   Returns Keywords



In [89]:

def GenerateCV(transcripts):
#   print("\n****** Transcript ******\n",transcripts[0],transcripts[1],transcripts[2],transcripts[3],transcripts[4],transcripts[5],transcripts[6],transcripts[7],sep='\n')
  transcripts[0]=set_name(transcripts[0])
  print("\nName Corrected:",transcripts[0])
  transcripts[2]=set_Address(transcripts[2])
  print("Address Corrected:",transcripts[2])
    
  #************************ TRANSLATION  ********************************
  translator = google_translator()  
  eng=[]
  for s in transcripts:
    translation = translator.translate(s,lang_tgt='en')  
    eng.append(translation)
  # i=0
  # for s in transcripts:
  #   print(s," " ,eng[i] )
  #   i+=1

  #************************ Keyword Extraction  ********************************
# name-0 age-1 address-2 phone-3 edu-4 year-5 prof-6 exp-7
#   print(transcripts,eng)
  name=eng[0]
  age=eng[1]
  address=eng[2]
  phone=eng[3]
  education=eng[4]
  year=eng[5]
  profession=eng[6]
  experience=eng[7] 

  print("\n****** Translation ******\n",name,age,address,phone,education,year,profession,experience,sep='\n')
  
  keywords=[]
# name-0 age-1 address-2 phone-3 edu-4 year-5 prof-6 exp-7
  keywords.append(get_name(name))
  keywords.append(get_age(age))
  keywords.append(get_address(address))
  keywords.append(get_phone(phone))
  keywords.append(get_education(education))
  keywords.append(get_year(year))
  keywords.append(get_profession(profession))
  keywords.append(get_experience(experience))
  
  print("\n****** Keywords ******\n",name,age,address,phone,education,year,profession,experience,sep='\n')
  cv_path="/home/baeg/Desktop/SpeakCv/cv_template.docx"
  #print(name,age,address,phone,education+" "+ year,profession,experience,sep='\n')
  Place_Keywords(keywords, cv_path)
  return keywords
# file_object.close()


## Calling asr(),and GenerateCV functions to get transcripts from trained model and generate CV

In [90]:
fileNames=['name','age',"address","phone","education","year","profession","experience"]
Transcripts=[]
loc='/home/baeg/Desktop/SpeakCv/Audios/'
wavCorrector()
for f in fileNames:
    
    l=loc+f+".wav"
    x=asr(l)
#     x=ASR_G(l)
    Transcripts.append(x)
GenerateCV(Transcripts)


/home/baeg/anaconda3/lib/python3.7/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/home/baeg/anaconda3/lib/python3.7/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


shape of specto  (437, 161)
Original: میرا نام سارا ہے
LM corrected: میرا نام سارا ہے
shape of specto  (419, 161)
Original: میری مر چالیس سال ہے
LM corrected: میری مر چالیس سال ہے
shape of specto  (431, 161)
Original: م اسلام باد سے ہوںے
LM corrected: م اسلام باد سے ہونے
shape of specto  (1091, 161)
Original: میرا فور نمبر ہے زیرو تری ون ون زیرو ون ٹو فور ٹری سس
LM corrected: میرا فور نمبر ہے زیرو تری ون ون زیرو ون ٹو فور ٹری سے
shape of specto  (437, 161)
Original: میں نے میٹرک کیا ہے
LM corrected: میں نے میٹرک کیا ہے
shape of specto  (641, 161)
Original: میں نے دو ہزار دس نے ویٹرک کیا تھے
LM corrected: میں نے دو ہزار دس نے میٹرک کیا تھے
shape of specto  (377, 161)
Original: میرہ پیشہ پلمب ہے
LM corrected: میرا پیشہ پلمبر ہے
shape of specto  (509, 161)
Original: میں نے دس سال کام کیا ہے
LM corrected: میں نے دس سال کام کیا ہے

Name Corrected: میرا نام سارہ ہے
Address Corrected: میں اسلام آباد سے ہے

****** Translation ******

My name is Sarah 
My dead is forty years 
I have from Islama

['Sarah',
 'forty years',
 'Islamabad',
 '03311234567',
 'matric',
 '2010',
 'plumber',
 'ten years']

In [91]:
# #NAMES
# words1=["Jawad", "Hassan", "Shafaq", "Riaz", "Bhatti", "Ejaz", "Ahmed", "Noreen", "Jamil", "Zainab", "Abaid", "Umair", "Arshad", "Madiha", "Umar", "Muhammad", "Toqeer", "talha", "mujahid", "Hamza", "ali", "alina", "baig", "fatima", "zainab", "Ahmed", "usman", "Anum", "Rana", "Sheikh", "mahrukh", "Bilal", "Ramsha", "zarish", "Waqas", "Sadaf", "Rashid", "ALIYA", "Salman", "Amna", "zeeshan", "Tabinda", "saad", "Ibrahim", "Zahid", "raja", "Amna", "Ayesha", "ismail", "Hammad", "saif", "Ahmad", "Yasir", "Muzammil", "Malik", "Eishal", "Hassan", "Waseem", "Amir", "Rabia", "Hussain", "sohail", "Sheraz", "Ayeza", "Ameer", "Inaya", "Kamran", "Fizza", "Rabab", "Abbas", "Khudija", "idrees", "khan", "sonia", "Adnan", "kashif", "Ghulam", "Qadir", "Dua", "sajida", "Uzair", "Huma", "sharif", "shaukat", "maheen", "alif", "khan", "Abd" , "ur rehman", "Ateeq", "ul rehman", "Shan", "Laiba", "Rizwan", "Murtaza", "Adeena", "HINA", "Zain", "Minahil", "Noor", "Zarak", "Amina", "Sidra", "hanzla", "Alishba", "Rohail", "sheeza", "Farrukh", "Saima", "Ayaan", "sahar", "Shazeb", "Rauf", "Shah", "Madhia", "jamshed", "Areej", "Ali", "Husnain", "Numan ", "javed", "Manaal", "Aruba", "luqman", "Rida", "asim", "Khadija", "Mudassir", "Maryam", "rahim", "sarmad", "Muneeb", "Naila", "Zohaib", "Sarah", "Shaheryar", "Annie", "Maaz", "Kashaf", "Gul", "Sher", "Humna", "Aaqib", "Amaan", "Mohid", "fazal", "Esan", "tariq", "Fahad", "Toqeer", "Zia", "mustafa", "Khawaja", "Shariq", "Muhammad", "khan", "Shahzad", "Hassam", "Waleed", "iqbal", "abrar", "Hasnain", "waqar", "faisal", "asif", ]
# words =["جواد", "حسن", "شفق" ,"ریاض" ,"بھٹی", "اعجاز" ,"احمد" ,"نورین" ,"جمیل" ,"زینب","عابد", "عمیر" ,"ارشد" ,"مدیحہ" ,"عمر", "محمد", "توقیر", "طلحہ", "مجاہد" ,"حمزہ" ,"علی" ,"علینہ", "بیگ" ,"جواد", "حسن", "شفق" ,"ریاض" ,"بھٹی" ,"اعجاز" ,"احمد", "نورین" ,"جمیل", "زینب" ,"عابد", "عمیر", "ارشد", "مدیحہ" ,"عمر", "محمد", "توقیر" ,"طلحہ", "مجاہد","حمزہ" ,"علی" ,"علینہ" ,"بیگ" ,"رباب", "عباس" , "ادریس" ,"خان" ,"سونیا", "عدنان", "کاشف", "غلام", "قادر", "دعا" ,"ساجدہ", "عزیر" ,"ہما", "شریف", "شوکت", "ماہین", "الف", "خان" ,"عبد" , "عتیق", "الرحمن" ,"شان", "لائبہ", "رضوان", "مرتضیٰ", "عدینہ","حنا", "زین", "مناہیل", "نور", "زرک" ,"آمنہ" ,"سدرہ", "ہنزلہ" ,"الیشبہ" ,"روہیل" ,"شیزہ", "فرخ", "صائمہ" ,"آیان" ,"سحر", "شازیب", "رؤف", "شاہ", "مدیحہ", "جمشید" ,"عریج",
#     "علی" ,"حسنین", "نعمان", "جاوید" ,"منال" ,"اروبا", "لقمان" ,"ردا", "عاصم" ,"خدیجہ" ,"مدثر", "مریم" ,"رحیم","سرمد" ,"منیب" ,"نائلہ" ,"زوہیب","سارہ" ,"شہر", "یار" ,"اینی", "معاذ", "کشاف" ,"گل", "شیر" ,"ہمنا", "عاقب", "امان" ,"موحد" ,"فضل", "ایسان" ,"طارق" ,"فہد","توقیر", "ضیا", "مصطفیٰ" ,"خواجہ", "شارق", "محمد", "خان", "شہزاد", "حسام", "ولید", "عقبال", "ابرار", "حسنین", "وقار", "فیصل" ,"آصف", "فاطمہ",
#     "زینب" "احمد", "عثمان", "انم" , "شیخ" ,"ماہرخ" ,"بلال" ,"رمشا" ,"زرش", "وقاص" ,"صدف" ,"راشد", "علیا" ,"سلمان", "آمنہ", "ذیشان" ,"تابندہ" ,"سعد", "ابراہیم" ,"زاہد" ,"راجہ", "آمنہ" ,"عائشہ", "اسماعیل" ,"حماد" ,"سیف" ,"احمد" ,
#     "یاسر" ,"مزمل" ,"ملک" ,"ایشال"  ,"وسیم" ,"عامر" ,"رابعہ" ,"حسین" ,"سہیل" ,"شیراز" ,"عائزہ" ,"عامر", "عنایہ" ,"کامران", "فزا"]
# len(words),len(words1)


# def city_Dist(sentence):
#     words =["جواد", "حسن", "شفق" ,"ریاض" ,"بھٹی", "اعجاز" ,"احمد" ,"نورین" ,"جمیل" ,"زینب"

#     ,"عابد", "عمیر" ,"ارشد" ,"مدیحہ" ,"عمر", "محمد", "توقیر", "طلحہ", "مجاہد" ,"حمزہ" ,"علی"]
#     words1=["Jawad", "Hassan", "Shafaq", "Riaz", "Bhatti", "Ejaz", "Ahmed", "Noreen", "Jamil", "Zainab",

#     "Abaid", "Umair", "Arshad", "Madiha", "Umar", "Muhammad", "Toqeer", "talha", "mujahid", "Hamza", "ali"]
#     len(words),len(words1)
# #     words =["کراچی", "لاہور", "فیصل", "آباد", "راول", "پنڈی","راولپنڈی", "گوجرانوالہ" ,"پشاور", "ملتان", "حیدرآباد" ,"اسلام آباد","اسلام", "آباد", "کوئٹہ" ,"بہاولپور" ,"بہاولپور", "سرگودھا", "سیالکوٹ"]
#     words=replaceUnicode(words)
#     words1=replaceUnicode(words1)
#     sentence = nltk.tokenize.word_tokenize(sentence)
#     sentence
#     listed = list()
#     for word in words:
#         for i in range(len(sentence)):
#           ed = similar(word,sentence[i])
#           listed.append(tuple([word,i, ed]))
#           listed.sort(key=lambda x:x[2],reverse=True)
#     nearest=listed[0][0]
#     index=listed[0][1]
#     if listed[1][1]-listed[0][1]==1: #abs(
#             nearest=listed[0][0]+listed[1][0]
#     print(nearest,index,words1[index])
#     return nearest,index 
# sentence="میرا نام لینا ہے"
# set_name(sentence)


In [92]:

# # sentence="میرا نام لینا ہے"
# # sentence="میں  لینا ہوں"
# # s="م اسلام باد سے ہونے"
# # set_name(sentence)
# # s=set_Address(s)
# s="میرا فور نمبر ہے زیرو تری ون ون زیرو ون ٹو فور ٹری سس"

# set_name(sentence)
# s=set_Phone(s)
# s
# translator = google_translator()  
# translate_text = translator.translate(s,lang_tgt='en')  
# get_phone(translate_text)

In [93]:
# # transcripts=ASR_G()
# Transcripts=["میرا نام حنان ہے","میری عمر 27 سال ہے","میں راولپنڈی میں رہتا ہوں","میرا فون نمبر یہ ہے 03001234567"," میں نے مڈل تک تعلیم حاصل کی ہے","میں نے 2010 میں میٹرک کیا"," میرا پیشہ نمبر ہے"," میرا پانچ سال کا تجربہ ہے"]
# GenerateCV(Transcripts)

In [94]:
# # Calling GenerateCV function here...
# location='/home/baeg/Desktop/SpeakCv/Audios/'
# keywords=GenerateCV(location)
# print("\n",keywords[0],keywords[1], keywords[2], keywords[3], keywords[4]+" "+ keywords[5], keywords[6], keywords[7],sep='\n')
# #       (     name,       age,        address,     phone,     education+" "  + year,         profession,  experience)

In [95]:
# sentence="My For The Number is Zero Tier Win Zero One to Tour Tree"
# sentence1=[]
# words = ["my" , "phone" , "number" , "is" , "zero" , "one" , "two" , "three", "four" , "five", "six", "seven", "eight", "nine"]
# sentence="میرا فور نمبر ہے زیرو تری ون ون زیرو ون ٹو فور ٹری سے"
# words = ["صفر","ایک","دو","تین","چار","پانچ","چھ","سات","آٹھ","نو","میرا","فون" ,"نمبر","زیرو","ون", "ٹو" ,"تھری" ,"فور", "فائیو" ,"سکس" ,"سیون", "آٹھ", "نائن"]  

# words =["کراچی", "لاہور", "فیصل", "آباد", "راول", "پنڈی","راولپنڈی", "گوجرانوالہ" ,"پشاور", "ملتان", "حیدرآباد" ,"اسلام آباد","اسلام", "آباد", "کوئٹہ" ,"بہاولپور" ,"بہاولپور", "سرگودھا", "سیالکوٹ"]



In [96]:

# get_address()
    # words =["‫میں‬","‫سے‬","‫ہے‬","‫تعلق‬","‫رہتا‬","‫ہوں‬","رہتی","‫میرا‬",           "کراچی", "لاہور", "فیصل", "آباد", "راول", "پنڈی","راولپنڈی", "گوجرانوالہ" ,"پشاور", "ملتان", "حیدرآباد" ,"اسلام آباد","اسلام", "آباد", "کوئٹہ" ,"بہاولپور" ,"بہاولپور", "سرگودھا", "سیالکوٹ"]
    # print(words)
# 
  # words = ["صفر","ایک","دو","تین","چار","پانچ","چھ","سات","آٹھ","نو","میرا","فون" ,"نمبر","زیرو","ون", "ٹو" ,"تری" ,"فور", "فائیو" ,"سکس" ,"سیون", "آٹھ", "نائن"]


In [97]:

# #      main,sub 
# sentence="لینا"
# type(name_Dist(sentence))
# # print(name)


In [98]:

# a="زینب"
# b='لینا'
# c="عمیر"
# d="علینہ"

# print(similar(b, a))
# print(similar(b, c))
# print(similar(b, d))

In [99]:
# city,index=city_Dist("م اسلام باد سے ہونے")


In [100]:
# LM corrected: میرا نام لینا ہے
# LM corrected: میری مر چالیس سال ہے
# LM corrected: م اسلام باد سے ہونے
# LM corrected: میرا فور نمبر ہے زیرو تری ون ون زیرو ون ٹو فور ٹری سے
# LM corrected: میں نے میٹرک کیا ہے
# LM corrected: میں نے دو ہزار دس نے میٹرک کیا تھے
# LM corrected: میرا پیشہ پلمبر ہے
# LM corrected: میں نے دس سال کام کیا ہے

In [101]:

# #################################### Keyword extraction functions ############################################
# def get_name(sentence):
#   tokens = nltk.tokenize.word_tokenize(sentence)
#   name=[]
#   for i in range(len(tokens)):
#     if (tokens[i]=='میرا'):
#       i+=1
#       if (tokens[i]=='نام'):
#         i+=1
#         name=tokens[i:]
#         for n in name:
#             if (name!="ہے"):
#                 x=name_Dist(name)
#                 sentence[i]=x
#             i+=1
#     elif (tokens[i]=='میں'):
#       i+=1
#       name=tokens[i:]
#       for n in name:
#             if (name!="ہو" and name!="ہوں" ):
#                 x=name_Dist(name)
#                 sentence[i]=x 
# #   str1=" "
# #   if len(name)==0:
# #     str1='None'
# #   else:
# #     str1=str1.join(sentence)
    
#   return sentence


# def get_age(sentence):
#   tokens = nltk.tokenize.word_tokenize(sentence)
#   age=[]

#   doc=nlp(sentence)
#   for X in doc.ents:
#     if(X.label_=='DATE'):
#       age.append(X.text)

#   if(len(age)==0):
#     age=['None']
    
#   str1=" "
#   if len(age)==0:
#     str1='None'
#   else:
#     str1=str1.join(age)
#   return str1

# def get_phone(sentence):
#   num = ""
#   for c in sentence:
#       if c.isdigit():
#           num = num + c
#   if num=="":
#     num='None'
#   return num


# def get_address(sentence):
#   tokens = nltk.tokenize.word_tokenize(sentence)
#     #what if users says something diff like mera ghar karachi main hain lekin islamabad main kaam kr raha hun  ("I live in Rawalpindi")
#     #POS tagger , NER
#   addr=[]
#   for i in range(len(tokens)):

#     #I live in Faisalabad
#     if (tokens[i].lower()=='live'):
#       i+=1
#       if (tokens[i].lower()=='in'):
#         i+=1
#       addr=tokens[i:]

#     #I belong to 
#     elif (tokens[i].lower()=='belong'):
#       i+=1
#       if (tokens[i].lower()=='to'):
#         i+=1
#       addr=tokens[i:]

#     #I am from 
#     elif (tokens[i].lower()=='from'):
#       i+=1
#       addr=tokens[i:]
#     else:
#       addr=tokens[i:]

#   addr[:2]
#   str1=" "
#   if len(addr)==0:
#     str1='None'
#   else:
#     str1=str1.join(addr)
#   return str1


# def get_education(sentence):
#   year=''
#   s=text2int(sentence) +' '     #convert year in words to number                                  
#   yr=re.findall("(\d{4})\D",s)
#   if (len(yr)!=0):      # IF THERE is any year mentioned, get it
#     year=yr[0]

#   tokens = nltk.tokenize.word_tokenize(sentence)
#   EDUCATION = ['1st', '2nd', '3rd', '4th', '5th', '6th', '7th', '8th', '9th', '10th','11th','12th',
#               'first','second','third','fourth','fifth','sixth','seventh','eight','ninth','tenth','elevnth','twelfth',
#               'inter','middle','intermediate','matriculation','matric','ICS','Diploma','FSC','FA',
#               'BA','BCom','B.Com',  'BTECH', 'B.TECH' , 'graduate', 'SSC', 'HSC' ,'illiterate' ]
#   edu=[]
#   for i in range(len(tokens)):
#     x=tokens[i].lower()
#     for i in range(len(EDUCATION)):
#       c=EDUCATION[i].lower()
#       if (nltk.edit_distance(c , x )<=1):
#           if (i<24):
#             c+=" grade"
#           edu.append(c)
#           break
#   if len(edu)==0:
#     str1='None'
#   else:
#     str1=str(edu[0])
#   return str1

# def get_year(sentence):
#     year=''
#     s=text2int(sentence) +' '     #convert year in words to number                                  
#     yr=re.findall("(\d{4})\D",s)
#     if (len(yr)!=0):      # IF THERE is any year mentioned, get it
#          year=str(max(yr))

#     return year


# def get_profession(sentence):
#   tokens = nltk.tokenize.word_tokenize(sentence)
#   proffession=["delivery","carpenter", "driver","electrician","plumber"]
#   #"I know plumber work"
#   #"I am a carpenter"
#   prof=[]
#   for i in range(len(tokens)):
#     #I live in Faisalabad
#     x=tokens[i].lower()
#     for c in proffession:
#       if (nltk.edit_distance(c , x )<3):
#           if (c=="delivery"):
#             c="delivery boy"
#           prof.append(c)
#   str1=" "
#   if len(prof)==0:
#     str1='None'
#   else:
#     str1=str1.join(prof) 
#   return str1


# def get_experience(line):
#   tokens = nltk.tokenize.word_tokenize(line)
#   exp=[]
#   doc=nlp(line)
#   for X in doc.ents:
#     if(X.label_=='DATE'):
#       exp.append(X.text)

#   if(len(exp)==0):
#     exp=['None']
    
#   str1=" "
#   if len(exp)==0:
#     str1='None'
#   else:
#     str1=str1.join(exp) 
  
#   return str1